## Why Generators

> * 제너레이터는 한 번에 하나씩 구성요소를 반환해주는 이터러블을 생성해주는 객체이다

> * 제너레이터의 주요 목적은 메모리를 절약하는 것이다

### 중첩 루프 효율적으로 벗어나기

In [5]:
def search_nested_bad(array, desired_value):
    coords = None
    for i, row in enumerate(array):
        for j, cell in enumerate(row):
            if cell == desired_value:
                coords = (i, j)
                break
        if coords is not None:
            break
    
    if coords is None:
        raise ValueError('{} not found'.format(desired_value))

In [6]:
def _iterate_array2d(array2d):
    for i, row in enumerate(array2d):
        for j, cell in enumerate(row):
            yield (i, j), cell

def search_nested(array, desired_value):
    try:
        coord = next(coord for (coord, cell) in _iterate_array2d(array) if cell == desired_value)
    except StopIteration:
        raise ValueError('{} not found'.format(desired_value))
        
    return coord
            

### 코루틴(Coroutine)

쓰레드(Threads): OS 혹은 런타임 환경이 쓰레드들간 switching을 주관함 (스케쥴러)  
코루틴(Coroutine): 프로그래머가 스스로 셋 포인트를 정해 언제 switching 될 지를 결정함

In [14]:
def print_name(prefix):
    print('Searching prefix: {}'.format(prefix))
    try:
        while True:
            name = (yield)
            if prefix in name:
                print(name)
    except GeneratorExit:
        print('Closing coroutine')
        
            
corou = print_name('Dear')
corou.__next__() # This will start execution of coroutine

corou.send('Deer Atul')
corou.send('Dear Atul')
corou.close()


Closing coroutine
Searching prefix: Dear
Dear Atul
Closing coroutine


#### 코루틴을 이용한 파이프라인 형성

In [16]:
def producer(sentence, next_coroutine): 
    ''' 
    Producer which just split strings and 
    feed it to pattern_filter coroutine 
    '''
    tokens = sentence.split(" ") 
    for token in tokens: 
        next_coroutine.send(token) 
    next_coroutine.close() 
  
def pattern_filter(pattern="ing", next_coroutine=None): 
    ''' 
    Search for pattern in received token  
    and if pattern got matched, send it to 
    print_token() coroutine for printing 
    '''
    print("Searching for {}".format(pattern)) 
    try: 
        while True: 
            token = (yield) 
            if pattern in token: 
                next_coroutine.send(token) 
    except GeneratorExit: 
        print("Done with filtering!!") 
  
def print_token(): 
    ''' 
    Act as a sink, simply print the 
    received tokens 
    '''
    print("I'm sink, i'll print tokens") 
    try: 
        while True: 
            token = (yield) 
            print(token) 
    except GeneratorExit: 
        print("Done with printing!") 
  
pt = print_token() 
pt.__next__() 
pf = pattern_filter(next_coroutine = pt) 
pf.__next__() 
  
sentence = "Bob is running behind a fast moving car"
producer(sentence, pf) 

Done with printing!
I'm sink, i'll print tokens
Searching for ing
running
moving
Done with filtering!!


In [8]:
"""Clean Code in Python - Chapter 7: Using Generators

> Methods of the Generators Interface.

"""
import time

from log import logger


class DBHandler:
    """Simulate reading from the database by pages."""

    def __init__(self, db):
        self.db = db
        self.is_closed = False

    def read_n_records(self, limit):
        return [(i, f"row {i}") for i in range(limit)]

    def close(self):
        logger.debug("closing connection to database %r", self.db)
        self.is_closed = True


def stream_db_records(db_handler):
    """Example of .close()

    >>> streamer = stream_db_records(DBHandler("testdb"))  # doctest: +ELLIPSIS
    >>> len(next(streamer))
    10

    >>> len(next(streamer))
    10
    """
    try:
        while True:
            yield db_handler.read_n_records(10)
            time.sleep(.1)
    except GeneratorExit:
        db_handler.close()


class CustomException(Exception):
    """An exception of the domain model."""


def stream_data(db_handler):
    """Test the ``.throw()`` method.

    >>> streamer = stream_data(DBHandler("testdb"))
    >>> len(next(streamer))
    10
    """
    while True:
        try:
            yield db_handler.read_n_records(10)
        except CustomException as e:
            logger.info("controlled error %r, continuing", e)
        except Exception as e:
            logger.info("unhandled error %r, stopping", e)
            db_handler.close()
            break
